# Inserimento manuale dei dati nel database
### Matter Of Design

In [27]:
pip install mysql-connector-python boto3

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# Librerie necessarie
import mysql.connector
import boto3
from tkinter import Tk, Label, Entry, Button, messagebox

## Inserimento dati

In [29]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Funzione per caricare un file su Amazon S3
def upload_to_s3(file_path, bucket_name, s3_key):
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_path, bucket_name, s3_key)
        return f"https://{bucket_name}.s3.amazonaws.com/{s3_key}"
    except Exception as e:
        print(f"Error uploading file: {e}")
        return None

# Funzione per inserire i dati nel database
def insert_data(codice_articolo, brand, finitura, posa, collezione, dimensione, 
                prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, 
                documento_path, immagine_path):
    cursor = db_connection.cursor()
    
    # Caricamento del documento su S3 (se fornito)
    documento_url = upload_to_s3(documento_path, 'matterofdesign-files', f'documenti/{codice_articolo}_doc') if documento_path else None
    immagine_url = upload_to_s3(immagine_path, 'matterofdesign-files', f'immagini/{codice_articolo}_img') if immagine_path else None

    query = """
    INSERT INTO Prodotto (codice_articolo, brand, finitura, posa, collezione, 
    dimensione, prezzo_unitario, prezzo_mq, colore, spessore, 
    tipologia_id, documento, immagine) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (codice_articolo, brand, finitura, 
              posa if posa else None,
              collezione, dimensione, 
              prezzo_unitario if prezzo_unitario is not None else None, 
              prezzo_mq if prezzo_mq is not None else None, 
              colore, 
              spessore if spessore is not None else None, 
              tipologia_id, 
              documento_url, immagine_url)
    cursor.execute(query, values)
    db_connection.commit()

# Funzione per raccogliere i dati dall'interfaccia e inserirli
def submit_data():
    try:
        codice_articolo = codice_articolo_entry.get()
        brand = brand_entry.get()
        finitura = finitura_entry.get()
        posa = posa_entry.get()
        collezione = collezione_entry.get()
        dimensione = dimensione_entry.get()
        prezzo_unitario = prezzo_unitario_entry.get()
        prezzo_mq = prezzo_mq_entry.get()
        prezzo_unitario = float(prezzo_unitario) if prezzo_unitario else None
        prezzo_mq = float(prezzo_mq) if prezzo_mq else None
        colore = colore_entry.get()
        spessore = spessore_entry.get()
        spessore = float(spessore) if spessore else None
        tipologia_id = int(tipologia_id_entry.get())
        documento_path = documento_entry.get()  # Percorso del file documento
        immagine_path = immagine_entry.get()  # Percorso del file immagine

        insert_data(codice_articolo, brand, finitura, posa, collezione, dimensione, 
                    prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, 
                    documento_path, immagine_path)
        messagebox.showinfo("Success", "Dati inseriti correttamente!")
    except Exception as e:
        messagebox.showerror("Error", f"Errore nell'inserimento dei dati: {e}")

# Connessione al database
db_connection = connect_to_db()

# Creazione dell'interfaccia grafica
root = Tk()
root.title("Inserimento Dati Prodotto")

# Creazione dei campi di input
Label(root, text="Codice Articolo").grid(row=0)
Label(root, text="Brand").grid(row=1)
Label(root, text="Finitura").grid(row=2)
Label(root, text="Posa").grid(row=3)
Label(root, text="Collezione").grid(row=4)
Label(root, text="Dimensione").grid(row=5)
Label(root, text="Prezzo Unitario").grid(row=6)
Label(root, text="Prezzo per MQ").grid(row=7)
Label(root, text="Colore").grid(row=8)
Label(root, text="Spessore").grid(row=9)
Label(root, text="Tipologia ID").grid(row=10)
Label(root, text="Percorso Documento").grid(row=11)
Label(root, text="Percorso Immagine").grid(row=12)

codice_articolo_entry = Entry(root)
brand_entry = Entry(root)
finitura_entry = Entry(root)
posa_entry = Entry(root)
collezione_entry = Entry(root)
dimensione_entry = Entry(root)
prezzo_unitario_entry = Entry(root)
prezzo_mq_entry = Entry(root)
colore_entry = Entry(root)
spessore_entry = Entry(root)
tipologia_id_entry = Entry(root)
documento_entry = Entry(root)
immagine_entry = Entry(root)

codice_articolo_entry.grid(row=0, column=1)
brand_entry.grid(row=1, column=1)
finitura_entry.grid(row=2, column=1)
posa_entry.grid(row=3, column=1)
collezione_entry.grid(row=4, column=1)
dimensione_entry.grid(row=5, column=1)
prezzo_unitario_entry.grid(row=6, column=1)
prezzo_mq_entry.grid(row=7, column=1)
colore_entry.grid(row=8, column=1)
spessore_entry.grid(row=9, column=1)
tipologia_id_entry.grid(row=10, column=1)
documento_entry.grid(row=11, column=1)
immagine_entry.grid(row=12, column=1)

# Pulsante per inviare i dati
Button(root, text="Inserisci", command=submit_data).grid(row=13, column=1)

# Avvio dell'interfaccia
root.mainloop()


Connected to MySQL database


## Verifica dati

In [30]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

def fetch_data():
    connection = connect_to_db()
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Prodotto")
    results = cursor.fetchall()
    for row in results:
        print(row)

# Esegui la funzione per vedere i dati
fetch_data()

Connected to MySQL database
('AL2487', 'Alice Ceramica', 'Lucido', 'Sospeso', 'Hide', '54x34', Decimal('388.00'), None, 'Bianco', None, 4, None, None)
